9. Суммируя весь предыдущий опыт напишите программу, которая будет скрейпить данные из imdb. Рекомендуемые библиотеки: BeautifulSoup, requests и random. Программа должна выполнять следующий функционал:

    a. При запуске программа должна подключаться к странице https://www.imdb.com/chart/top

    b. Собирать список фильмов (и по вашему желанию их описание)

    c. Выводить 10 случайных фильмов

Работает только на первые 25 фильмов из-за ленивой подгрузки. Для возможност парсить все 250 фильмов была идея использовать Selenium, однако не имею в нём большого опыта

In [14]:
%pip install -r requirements.txt
%pip install python-dotenv
# python-dotenv нужен для безопасного хранения API ключа

Note: you may need to restart the kernel to use updated packages.



In [2]:
import requests
from bs4 import BeautifulSoup
import random
from IPython.display import HTML, display
import re
import os
from dotenv import load_dotenv
from tqdm import tqdm

In [3]:
load_dotenv()
RAPIDAPI_KEY = os.getenv('RAPIDAPI_KEY')

In [4]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}
session = requests.session()

In [5]:
def get_discription(url):
    html = session.get(url, headers=headers).text
    soup = BeautifulSoup(html, 'lxml')
    span = soup.find('span', {'data-testid': 'plot-xl', 'role': 'presentation'})
    if span:
        return span.text.strip()
    else:
        return "No description found"

In [6]:
def deep_translate(text):
    if not RAPIDAPI_KEY:
        return ''
    url = 'https://deep-translate1.p.rapidapi.com/language/translate/v2'
    payload = {
        'q': text,
        'source': 'en',
        'target': 'ru'
    }
   
    headers = {
        'content-type': 'application/json',
        'X-RapidAPI-Key': RAPIDAPI_KEY,
        'X-RapidAPI-Host': 'deep-translate1.p.rapidapi.com'
    }
    try:
        return requests.post(url, json=payload, headers=headers).json()['data']['translations']['translatedText']
    except KeyError:
        return '<ошибка во время перевода>'

In [7]:
class Movie:
    def __init__(self, poster_url, title, year, rating, description=''):
        self.poster_url = poster_url
        self.title = title
        self.year = year
        self.rating = rating
        self.description = description
    
    @classmethod
    def from_li_str(cls, li_str):
        soup = BeautifulSoup(li_str, 'lxml')
        poster_url = soup.find('img')['src']
        title_full = soup.find('h3', class_='ipc-title__text').text.strip()
        title = re.sub(r'^\d+\.\s*', '', title_full)
        year = soup.find('span', class_='cli-title-metadata-item').text.strip()[:4]
        rating = soup.find('span', class_='ipc-rating-star--imdb').text.strip()[:3]
        discr_url = fr'https://www.imdb.com/{soup.find('a', class_='ipc-title-link-wrapper').get('href')}'
        discription = get_discription(discr_url)
        return cls(poster_url, title, year, rating, discription)

    def get_dict(self):
        return {'poster': self.poster_url, 
                'title': self.title, 
                'year': self.year, 
                'rating': self.rating, 
                'description': self.description}
    
    def display_movie(self):
        html = f"""
        <div style="display: flex; flex-direction: column;">
            <div style="display: flex; flex-direction: row;">
                <div style="margin-right: 20px;">
                    <img src="{self.poster_url}" alt="{self.title}">
                </div>
                <div style="text-align: center;">
                    <h3>{self.title}</h3>
                    <p>Год: <strong>{self.year}</strong></p>
                    <p>Оценка IMDb: <strong>{self.rating}</strong></p>
                </div>
            </div>
            <div style="text-align: justify;">
                <h4>Краткое описание</h4>
                <p>{self.description}</p>
                <p>{deep_translate(self.description)}</p>
            </div>
        </div>
        """
        display(HTML(html))

In [8]:
url = 'https://www.imdb.com/chart/top/'

In [9]:
response = session.get(url, headers=headers)
response.raise_for_status()

In [10]:
soup = BeautifulSoup(response.text, 'lxml')
movies_html = soup.find('ul', role='presentation')

In [15]:
movies = []
for movie_html in tqdm(movies_html.find_all('li'), colour='green'):
    movies.append(Movie.from_li_str(str(movie_html)))

100%|██████████| 25/25 [00:32<00:00,  1.29s/it]


In [16]:
random_movies = random.sample(movies, 10)
for movie in random_movies:
    movie.display_movie()